**Generative AI - Assignment 4**

**Sarah Groark**

Train Generative Adversarial Network on dataset consisting of images of bedrooms.

======================================================================

This notebook consists of the code for processing the data, defining the network, and establishing the code for GIF creation.

In [ ]:
#import libraries
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time
from IPython import display

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()

    # Initial Dense layer to create a base for reshaping
    model.add(layers.Dense(8 * 8 * 512, use_bias=False, input_shape=(100,)))  # Increased to 512
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Reshape to (8, 8, 512)
    model.add(layers.Reshape((8, 8, 512)))
    assert model.output_shape == (None, 8, 8, 512)

    # First transposed convolution to upscale to (16, 16, 256)
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))  # Increased to 256
    assert model.output_shape == (None, 16, 16, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Second transposed convolution to upscale to (32, 32, 128)
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))  # Increased to 128
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Third transposed convolution to upscale to (64, 64, 64)
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))  # Increased to 64
    assert model.output_shape == (None, 64, 64, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Output layer to generate (64, 64, 3) color images
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 64, 64, 3)

    return model

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# download dataset
!kaggle datasets download -d jhoward/lsun_bedroom

Dataset URL: https://www.kaggle.com/datasets/jhoward/lsun_bedroom
License(s): ODbL-1.0
100% 8.89G/8.89G [00:47<00:00, 82.7MB/s]
100% 8.89G/8.89G [00:47<00:00, 199MB/s] 


In [ ]:
# create folder to store unzipped images
images_folder = '/content/unzipped_images'
os.makedirs(images_folder, exist_ok=True)

In [ ]:
import zipfile

zip_file_path = '/content/lsun_bedroom.zip'
extract_to = '/content/unzipped_images'

# Extract the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Verify the extracted contents
os.listdir(extract_to)

# List the contents of the 'content' directory
os.listdir('/content/unzipped_images')

['sample', 'data0']

In [ ]:
from PIL import Image

# loads + processes a single image

def load_image(file_path):
    image = Image.open(file_path)
    image = image.resize((64, 64))
    image = np.array(image) / 127.5 - 1
    return image

# generator function

def image_generator(image_paths):
    for path in image_paths:
        yield load_image(path)

root_folder = '/content/unzipped_images/data0/lsun/bedroom'

# create list of paths
image_paths = []

for root, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))

# take subset of only 5000 images

subset_images = image_paths[:5000]

train_dataset = tf.data.Dataset.from_generator(
    lambda: image_generator(subset_images),
    output_signature=tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32)
)

BUFFER_SIZE = len(subset_images)
BATCH_SIZE = 512

train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# store array of subsetted training images
train_images = np.array([load_image(path) for path in subset_images])

In [ ]:
#import libraries
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time
from IPython import display

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()

    # First convolutional layer
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(64, 64, 3)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Second convolutional layer
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Third convolutional layer
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Flatten and output layer
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
# LOSS FUNCTION
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

#define learning rates
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
EPOCHS = 200
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# Visualization function
output_dir = 'generated_images'
os.makedirs(output_dir, exist_ok=True)

def plot_images(real_images, generated_images, epoch):
    plt.figure(figsize=(12, 6))

    # Plot real images
    plt.subplot(1, 2, 1)
    plt.title('Real Images')
    for i in range(min(5, len(real_images))):
        plt.imshow((real_images[i] + 1) / 2)  # Rescale to [0, 1]
        plt.axis('off')

    # Plot generated images
    plt.subplot(1, 2, 2)
    plt.title('Generated Images at Epoch {}'.format(epoch))
    for i in range(min(5, len(generated_images))):
        plt.imshow((generated_images[i] + 1) / 2)  # Rescale to [0, 1]
        plt.axis('off')

    # Save image
    plt.savefig(os.path.join(output_dir, f'epoch_{epoch}.png'))
    plt.show()

def train(dataset, epochs, train_images):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Generate images for visualization
        noise = tf.random.normal([5, noise_dim])
        generated_images = generator(noise, training=False)
        plot_images(train_images, generated_images, epoch + 1)

        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print('Generator Loss: ', gen_loss.numpy(), ' Discriminator Loss: ', disc_loss.numpy())
        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))

    # Generate after the final epoch
    generate_and_save_images(generator, epochs, seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
def create_gif(output_dir, gif_filename, duration=0.5):
    # Get saved generated images from the output directory
    filenames = glob.glob(os.path.join(output_dir, 'epoch_*.png'))
    filenames.sort(key=lambda f: int(os.path.splitext(os.path.basename(f))[0].split('_')[1]))  # Sort by epoch number

    # Create a GIF
    with imageio.get_writer(gif_filename, mode='I', duration=duration) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

# create the GIF
output_dir = '/content/generated_images'
gif_filename = 'dcgan.gif'
create_gif(output_dir, gif_filename)

<ipython-input-26-0a370e4a37d1>:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [ ]:
!pip install git+https://github.com/tensorflow/docs